In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
df1 = pd.read_excel('/content/FinalHinTrainTrac2_BackTran(clean).xlsx', index_col = 0) 
df2 = pd.read_excel('/content/FinalHinDevTrac2_BackTran(clean).xlsx', index_col = 0)

In [ ]:
df1.head()

,ID,data,Sub-task A,Sub-task B
0,C4.131,बॉलीवुड फिल्म देखने के समय तर्क घर में चोरके ए...,NAG,NGEN
1,C4.638,चुटिया फिल्म,NAG,NGEN
2,C38.598,हमारे जाट बन्दे का खेने का अर्थ था मार्च डालुग...,OAG,NGEN
3,C4.2101.1,फेमिनिस्म कैंसर की नारीवादी ये साही हैं संयुक...,OAG,NGEN
4,C29.14.2,अमृत आनंद अब तो जुड़े ही है उनको बोलो जुड़ने,NAG,NGEN


In [ ]:
df2.head()

,ID,data,Sub-task A,Sub-task B
0,C38.9,बक्ल साक्षात्कारकर्ताओं कुछ जाड़ा हाय खुले दिम...,OAG,NGEN
1,C4.1510,भैया शादी मैं ज़रूर आना फिल्म का कथानक था क्या...,NAG,NGEN
2,C19.95,धारा हैं क्या ये बताते कम से कम को,NAG,NGEN
3,C4.281,कबीर सिंह मारा हुई इससे पता चलता है आजकल के लो...,OAG,NGEN
4,C4.216,मेन इतनी कम नापसंद क्भी न्ही देखी,NAG,NGEN


In [ ]:
df1['Sub-task A'].value_counts()

CAG    2453
NAG    2245
OAG    1820
Name: Sub-task A, dtype: int64

In [ ]:
df2['Sub-task A'].value_counts()

CAG    602
OAG    582
NAG    578
Name: Sub-task A, dtype: int64

In [ ]:
df = pd.concat([df1, df2], ignore_index = True)
df['Sub-task A'].value_counts()

CAG    3055
NAG    2823
OAG    2402
Name: Sub-task A, dtype: int64

In [ ]:
index = df.index
len(index)

8280

In [ ]:
samples = df['data']
labels = df['Sub-task A']

In [ ]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [ ]:
max_len = np.max([len(text.split()) for text in train_samples])
max_len

508

In [ ]:
train_labels = list(train_labels)
val_labels = list(val_labels)

In [ ]:
class_names = list(df['Sub-task A'].unique())
class_names

['OAG', 'CAG', 'NAG']

In [ ]:
class_index = [0, 1, 2] #NAG, CAG, OAG
i=0
for y in train_labels:
  if y=='NAG': train_labels[i] = 0
  elif y=='CAG': train_labels[i] = 1
  elif y == 'OAG': train_labels[i] = 2
  i=i+1

i=0
for y in val_labels:
  if y=='NAG': val_labels[i] = 0
  elif y=='CAG': val_labels[i] = 1
  elif y == 'OAG': val_labels[i] = 2
  i=i+1

In [ ]:
print(train_labels)
print(val_labels)

[2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 0, 1, 0, 2, 1, 2, 1, 0, 0, 2, 0, 1, 1, 0, 2, 0, 1, 1, 0, 1, 1, 2, 0, 2, 1, 0, 2, 2, 1, 1, 2, 0, 1, 0, 1, 2, 0, 1, 2, 2, 1, 2, 0, 2, 0, 1, 2, 2, 0, 1, 1, 1, 1, 1, 0, 2, 1, 2, 1, 2, 1, 1, 2, 1, 0, 2, 2, 1, 0, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 0, 2, 2, 1, 0, 0, 1, 1, 2, 2, 2, 2, 2, 1, 0, 2, 0, 1, 1, 0, 0, 1, 0, 1, 2, 1, 0, 0, 2, 0, 0, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 0, 0, 1, 2, 2, 2, 0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 1, 1, 2, 2, 0, 0, 2, 2, 0, 2, 1, 1, 2, 2, 2, 2, 1, 0, 0, 2, 0, 0, 0, 1, 0, 2, 2, 0, 0, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 0, 0, 2, 1, 1, 1, 1, 0, 1, 1, 2, 0, 1, 1, 1, 1, 2, 0, 2, 1, 0, 1, 0, 0, 2, 2, 1, 1, 2, 0, 1, 2, 1, 0, 0, 2, 2, 0, 1, 1, 0, 1, 2, 1, 1, 2, 0, 2, 1, 1, 0, 1, 1, 2, 1, 0, 1, 1, 1, 2, 0, 1, 2, 1, 0, 1, 1, 2, 2, 1, 2, 0, 0, 2, 1, 2, 2, 2, 1, 2, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 0, 1, 0, 2, 0, 1, 1, 1, 2, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 2, 0, 0, 1, 1, 2, 1, 2, 0, 1, 0, 1, 0, 2, 2, 0, 2, 2, 1, 1, 2, 2, 

# Creating a vocabulary

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [ ]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'हैं', 'के', 'है']

In [ ]:
output = vectorizer(["बिल्ली चटाई पर बैठ गयी"])
output.numpy()[0, :5]

array([1470,  186,   42, 2637,  852])

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
test = "बिल्ली चटाई पर बैठ गयी"
test = test.split()
print(test)
[word_index[w] for w in test]

['बिल्ली', 'चटाई', 'पर', 'बैठ', 'गयी']


[1470, 186, 42, 2637, 852]

# Loading Pre Trained Vectors

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
word_vector = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/indicnlp.v1.hi.vec')

In [ ]:
vocab = word_vector.vocab

In [ ]:
embeddings_index = {}
for k in vocab.keys():
  embeddings_index[k] = np.array([word_vector.word_vec(k)])

In [ ]:
embeddings_index['गाड़ी'].shape

(1, 300)

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 300
hits = 0
misses = 0
missed_word = []
# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
        missed_word.append(word)
print("Converted %d words (%d misses)" % (hits, misses))

Converted 9625 words (2671 misses)


In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
from tensorflow.keras import layers
from keras.models import Input, Model
from keras.layers import LSTM, Dense, Embedding, concatenate, Dropout, concatenate
from keras.layers import Bidirectional

In [ ]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = Bidirectional(LSTM(256, return_sequences=True))(embedded_sequences)
x = Bidirectional(LSTM(150))(x)
x = layers.Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
x = layers.Dense(64, activation="relu")(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    3689400     input_1[0][0]                    
__________________________________________________________________________________________________
tf.compat.v1.shape_2 (TFOpLambd (3,)                 0           embedding[0][0]                  
__________________________________________________________________________________________________
tf.compat.v1.shape (TFOpLambda) (3,)                 0           embedding[0][0]                  
______________________________________________________________________________________________

In [ ]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [ ]:
print(y_train)
print(y_val)

[2 1 1 ... 2 1 2]
[2 1 1 ... 2 2 0]


In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
52/52 [==============================] - 152s 3s/step - loss: 1.0302 - acc: 0.4796 - val_loss: 0.9889 - val_acc: 0.5085
Epoch 2/20
52/52 [==============================] - 151s 3s/step - loss: 0.9572 - acc: 0.5451 - val_loss: 0.9661 - val_acc: 0.5302
Epoch 3/20
52/52 [==============================] - 154s 3s/step - loss: 0.9271 - acc: 0.5676 - val_loss: 0.9567 - val_acc: 0.5459
Epoch 4/20
52/52 [==============================] - 150s 3s/step - loss: 0.9140 - acc: 0.5785 - val_loss: 0.9460 - val_acc: 0.5417
Epoch 5/20
52/52 [==============================] - 149s 3s/step - loss: 0.9031 - acc: 0.5868 - val_loss: 0.9369 - val_acc: 0.5525
Epoch 6/20
52/52 [==============================] - 151s 3s/step - loss: 0.8930 - acc: 0.5876 - val_loss: 0.9440 - val_acc: 0.5574
Epoch 7/20
52/52 [==============================] - 154s 3s/step - loss: 0.8857 - acc: 0.5934 - val_loss: 0.9305 - val_acc: 0.5568
Epoch 8/20
52/52 [==============================] - 149s 3s/step - loss: 0.8740 - a